1.2. Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizarás la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas, en este caso IMDB, y extraer detalles como calificaciones, actores y directores.

Necesitamos la siguiente información:

Puntuación de IMDB (en caso de que la tenga).

Dirección (director/a o directores/as de cada película).

Guionistas (de cada película).

Argumento.

Nombre de la película.

Id de la película.

Bonus: También puedes extraer la siguiente información:

Duración de la película.

Puntuación de Rotten Tomatoes.

In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) 

PASOS

1. Generar driver
2. Llamamos a una funcion que recibe un navegador y un identificador de película y devuelve los datos de esa pelicula
3. Hacemos un bucle que recorra los identificadores de las peliculas que queremos explorar. 


In [2]:
# 1 GENERAR DRIVER

def generar_driver ():

    #Inicializamos el navegador.
    driver = webdriver.Chrome()

    sleep(2)

    #Maximizamos pantalla.
    driver.maximize_window()
    print("Maximizando la web.")

    sleep(2)


    #Navegamos a la página web "https://www.imdb.com/".
    driver.get("https://www.imdb.com/")
    print("Accediendo a la web.")
 

    sleep(2)

    #Aceptamos cookies.
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    print("Cookies aceptadas.")

    sleep(2)

    return driver


# mi_driver = generar_driver()
# type(mi_driver)



En lugar de pasarle el nombre de la película por un input, se lo pasamos por parámetro a la funcion.

Además, le vamos a pasar el driver por parámetro para que no tenga que estar entrando desde cero a IMDB con cada película.
👇

In [20]:
  
# Función para extraer los datos de una película usando su ID de IMDb
def extraer_datos_pelicula(driver, id_pelicula):
    # Construimos la URL de la película en IMDb usando su ID
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"
    driver.get(url_pelicula)
    print(f"Accediendo a la película con ID {id_pelicula}")

    sleep(2)

    #Ahora estamos dentro de la info de la película. Vamos a sacar los datos que se nos piden:

    #Nombre de la película.
    titulo_pelicula = driver.find_element("css selector", "h1").text
    print("Nombre de la película:", titulo_pelicula)

    #Puntuación de IMDb.
    try:
        puntuacion_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div.sc-3a4309f8-0.jJkxPn.sc-1f50b7c-1.doVJqQ > div > div:nth-child(1) > a > span > div > div.sc-eb51e184-0.ghvwpw > div.sc-eb51e184-2.fAhXAe > span.sc-eb51e184-1.ljxVSS").text
    except:
        puntuacion_pelicula = "No disponible"
    print("Puntuación de IMDb:", puntuacion_pelicula)

    #Info dirección.
    try:
        directores = driver.find_elements("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(1) > div")
        nombres_directores = [director.text.strip() for director in directores]
        print("Dirección:", ", ".join(nombres_directores))
        # Hay que guardar esto en una variable que luego le pasaremos a la tupla. 
        dato_direccion = ", ".join(nombres_directores)
        
    except:
        print("No se pudo encontrar la información de los directores.")

    #Info guionistas.
    try:
        guionistas = driver.find_elements("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(2) > div")
        nombre_guionistas = [guionista.text.strip() for guionista in guionistas]
        print("Guionistas:", ", ".join(nombre_guionistas)) #lo uso para separar nombres con comas 
        # Hay que guardar esto en una variable que luego le pasaremos a la tupla. 
        dato_guion = ",".join(nombre_guionistas) 

    except:
        print("No se pudo encontrar la información de los guionistas.")

    #Info argumento.
    try:
        argumento_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > p > span.sc-2d37a7c7-2.ggeRnl").text
    except:
        argumento_pelicula = "No disponible"
    print("Argumento:", argumento_pelicula)

    # ID de la película (sacarlo de la URL)
    id_pelicula = driver.current_url.split('/')[4]
    print("ID de la película:", id_pelicula)

    sleep(4)

    # Devolvemos los resultados que hemos obtenido en una tupla

    tupla_datos_pelicula = (titulo_pelicula, puntuacion_pelicula, dato_direccion, dato_guion, argumento_pelicula, id_pelicula)
    return tupla_datos_pelicula




In [21]:

ids_peliculas = ["tt0107290", "tt0076759", "tt0088763"]
mi_driver = generar_driver()

#Lista para almacenar los datos
datos_peliculas = []

for ids in ids_peliculas:
    mis_datos = extraer_datos_pelicula(mi_driver, ids)
    datos_peliculas.append(mis_datos)

mi_driver.quit()

# Crear un DataFrame con los datos extraídos
columnas = ["Título", "Puntuación IMDb", "Directores", "Guionistas", "Argumento", "ID"]
df_peliculas = pd.DataFrame(datos_peliculas, columns=columnas)

# Mostrar el DataFrame
print(df_peliculas)

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Accediendo a la película con ID tt0107290
Nombre de la película: Jurassic Park (Parque Jurásico)
Puntuación de IMDb: 8,2
Dirección: Steven Spielberg
Guionistas: Michael CrichtonDavid Koepp
Argumento: Gracias al ADN fosilizado en ámbar, John Hammond da vida a varias especies de dinosaurios y crea Jurassic Park, un parque temático en una isla de Costa Rica. Pero lo que parecía un sueño se convierte rápidamente en pesadilla.
ID de la película: tt0107290
Accediendo a la película con ID tt0076759
Nombre de la película: La guerra de las galaxias
Puntuación de IMDb: 8,6
Dirección: George Lucas
Guionistas: George Lucas
Argumento: Luke Skywalker une sus fuerzas con un caballero jedi, un piloto fanfarrón, un wookiee y dos droides para salvar a la galaxia de la estación espacial del Imperio, a la vez que intenta rescatar a la princesa Leia del malvado Darth Vader.
ID de la película: tt0076759
Accediendo a la película con ID tt0088763
Nom

In [23]:
df_peliculas.head()

,Título,Puntuación IMDb,Directores,Guionistas,Argumento,ID
0,Jurassic Park (Parque Jurásico),"8,2",Steven Spielberg,Michael CrichtonDavid Koepp,"Gracias al ADN fosilizado en ámbar, John Hammo...",tt0107290
1,La guerra de las galaxias,"8,6",George Lucas,George Lucas,Luke Skywalker une sus fuerzas con un caballer...,tt0076759
2,Regreso al futuro,"8,5",Robert Zemeckis,Robert ZemeckisBob Gale,"Marty McFly, un estudiante de secundaria de 17...",tt0088763
